In [129]:
import pandas as pd
import numpy as np

In [130]:
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns" ,None)

In [162]:
df = pd.read_csv("flat_houses_v6.csv")

# Similar Properties

In [132]:
df_rec = df.copy()
localities = df_rec["LOCALITY_NAME"].value_counts()[0:150].index
df_rec.loc[~df_rec["LOCALITY_NAME"].isin(localities), "LOCALITY_NAME"] = "other"
df_rec = df_rec[["PROP_ID", "PROPERTY_TYPE","LOCALITY_NAME", "BEDROOM_NUM", "BATHROOM_NUM","AGE_POSSESSION", "CALCULATED_AREA_SQFT"]]
df_rec.index = df_rec["PROP_ID"]
df_rec.drop(columns = ["PROP_ID"], inplace = True)

In [133]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [134]:
# Define columns for different encodings
one_hot_cols = ['PROPERTY_TYPE', 'LOCALITY_NAME','AGE_POSSESSION']
numeric_cols = ['BEDROOM_NUM', 'BATHROOM_NUM', 'CALCULATED_AREA_SQFT']

# Create transformers for each encoding type
preprocessor = ColumnTransformer(
    transformers=[
        ('one_hot', OneHotEncoder(drop = 'first'), one_hot_cols),
        ('scaler', StandardScaler(), numeric_cols)
    ],
    remainder = 'passthrough'
  )
# Create a pipeline that applies the transformers
pipeline = Pipeline([
    ('preprocessor', preprocessor)
])
# Fit and transform the DataFrame using the pipeline
df_encoded = pipeline.fit_transform(df_rec)

In [136]:
df_locality = df_encoded[:,0:150]
df_w = df_encoded[:, 151:]

In [141]:
import joblib

joblib.dump(df, 'df_v6.pkl', compress=('zlib', 3))
joblib.dump(df_locality, 'df_locality.pkl', compress = ('zlib', 3))
joblib.dump(df_w, 'df_w.pkl', compress = ('zlib', 3))
joblib.dump(pipeline, 'pipeline_encode.pkl' , compress = ('zlib',3))

['pipeline_encode.pkl']

In [145]:
data = [['flat', 'Azadpur', 'New Property', 2, 1, 500]]
columns = [['PROPERTY_TYPE', 'LOCALITY_NAME', 'AGE_POSSESSION', 'BEDROOM_NUM', 'BATHROOM_NUM', 'CALCULATED_AREA_SQFT']]

In [146]:
# Convert to DataFrame
one_df = pd.DataFrame(data, columns=columns)
one_df

,PROPERTY_TYPE,LOCALITY_NAME,AGE_POSSESSION,BEDROOM_NUM,BATHROOM_NUM,CALCULATED_AREA_SQFT
0,flat,Azadpur,New Property,2,1,500


In [147]:
df_encoded_one_df = pipeline.transform(one_df)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [148]:
df_locality_one_df = df_encoded_one_df[:,0:150]

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [149]:
cosine_sim_locality = cosine_similarity(df_locality, df_locality_one_df)
similarity_locality_df = pd.DataFrame(cosine_sim_locality, index = df_rec.index)
similarity_locality_df.rename(columns = {0: "similarity_locality_score"}, inplace = True)
cosine_sim = cosine_similarity(df_encoded[:, 151:], df_encoded_one_df[:, 151:])
similarity_df = pd.DataFrame(cosine_sim, index = df_rec.index)
similarity_df.rename(columns = {0: "similarity_df_score"}, inplace = True)

In [150]:
new_df = pd.merge(similarity_df, similarity_locality_df, on = "PROP_ID", how = "inner")
new_df["similarity_score"] = 0.6*new_df["similarity_locality_score"] + 0.4*new_df["similarity_df_score"]
new_df = new_df["similarity_score"]

In [152]:
l1 = new_df.sort_values(ascending = False)[0:5].index.values.tolist()

In [ ]:
# l1 contains the list of top 5 similar properties ID



---



---



# Nearest Properties

In [ ]:
!pip install haversine

In [ ]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

def haversine_distance(lat1, lon1, lat2, lon2):
    import math

    # Radius of the Earth in kilometers
    R = 6371.0

    # Convert latitude and longitude from degrees to radians
    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)

    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c

    return distance

# Calculate distances using the Haversine function
dist_matrix = pdist(df1[['LATITUDE', 'LONGITUDE']], metric=lambda u, v: haversine_distance(u[0], u[1], v[0], v[1]))

# Convert the distance matrix to a square DataFrame
distances_df = pd.DataFrame(squareform(dist_matrix), index=df1['PROP_ID'], columns=df1['PROP_ID'])



In [ ]:
distances_df.head()

PROP_ID    G71293060  Y71293054  D71293050  B71293046  A71293042  U69323800  \
PROP_ID                                                                       
G71293060        0.0        0.0        0.0        0.0        0.0   7.073408   
Y71293054        0.0        0.0        0.0        0.0        0.0   7.073408   
D71293050        0.0        0.0        0.0        0.0        0.0   7.073408   
B71293046        0.0        0.0        0.0        0.0        0.0   7.073408   
A71293042        0.0        0.0        0.0        0.0        0.0   7.073408   

PROP_ID    B70962206  A70962204  Y70962202  W71303626  M71303620  E69472490  \
PROP_ID                                                                       
G71293060  16.249363  16.249363  16.249363  16.346066  16.346066  12.458451   
Y71293054  16.249363  16.249363  16.249363  16.346066  16.346066  12.458451   
D71293050  16.249363  16.249363  16.249363  16.346066  16.346066  12.458451   
B71293046  16.249363  16.249363  16.249363  16.346066  16.346066  12.458451   
A71293042  16.249363  16.249363  16.249363  16.346066  16.346066  12.458451   

PROP_ID    A71264074  W71264072  P71264070  O71264068  X71264066  P71264064  \
PROP_ID                                                                       
G71293060        0.0        0.0        0.0        0.0        0.0        0.0   
Y71293054        0.0        0.0        0.0        0.0        0.0        0.0   
D71293050        0.0        0.0        0.0        0.0        0.0        0.0   
B71293046        0.0        0.0        0.0        0.0        0.0        0.0   
A71293042        0.0        0.0        0.0        0.0        0.0        0.0   

PROP_ID    Y70632738  Y71293566  H71293564  J71293556  I70297564  E70580962  \
PROP_ID                                                                       
G71293060  10.414835   4.285075   4.285075   4.285075  17.739844  11.187043   
Y71293054  10.414835   4.285075   4.285075   4.285075  17.739844  11.187043   
D71293050  10.414835   4.285075   4.285075   4.285075  17.739844  11.187043   
B71293046  10.414835   4.285075   4.285075   4.285075  17.739844  11.187043   
A71293042  10.414835   4.285075   4.285075   4.285075  17.739844  11.187043   

PROP_ID    Q69200278  B68361094  M68361090  S70168030  V70168028  C70168024  \
PROP_ID                                                                       
G71293060   7.995444   4.266147   4.266147  11.571975  11.571975  11.571975   
Y71293054   7.995444   4.266147   4.266147  11.571975  11.571975  11.571975   
D71293050   7.995444   4.266147   4.266147  11.571975  11.571975  11.571975   
B71293046   7.995444   4.266147   4.266147  11.571975  11.571975  11.571975   
A71293042   7.995444   4.266147   4.266147  11.571975  11.571975  11.571975   

PROP_ID    G70150382  F70150378  Q70150374  X70538000  F68731988  E68731986  \
PROP_ID                                                                       
G71293060  14.077206  14.077206  14.077206  15.011827  15.949172  15.949172   
Y71293054  14.077206  14.077206  14.077206  15.011827  15.949172  15.949172   
D71293050  14.077206  14.077206  14.077206  15.011827  15.949172  15.949172   
B71293046  14.077206  14.077206  14.077206  15.011827  15.949172  15.949172   
A71293042  14.077206  14.077206  14.077206  15.011827  15.949172  15.949172   

PROP_ID    R68731982  H68731980  T70644280  P71271554  S71182476  N71182474  \
PROP_ID                                                                       
G71293060  15.949172  15.949172  15.711395  12.122729  17.848886  17.848886   
Y71293054  15.949172  15.949172  15.711395  12.122729  17.848886  17.848886   
D71293050  15.949172  15.949172  15.711395  12.122729  17.848886  17.848886   
B71293046  15.949172  15.949172  15.711395  12.122729  17.848886  17.848886   
A71293042  15.949172  15.949172  15.711395  12.122729  17.848886  17.848886   

PROP_ID    G67235554  V70221608  A69434640  I69719724  H71344076  O71146016  \
PROP_ID                                    

In [ ]:
df[df["LOCALITY_NAME"] == 'Sector 19 Rohini']

,PROPERTY_TYPE,CITY_NAME,LOCALITY_NAME,OWNTYPE,BEDROOM_NUM,BATHROOM_NUM,BALCONY_NUM,FURNISH,LATITUDE,LONGITUDE,PRICE_CR,CALCULATED_AREA_SQFT,AGE_POSSESSION,AMENITIES_CLUSTER,FLOOR_CATEGORY
PROP_ID,,,,,,,,,,,,,,,
K71113110,flat,North Delhi,Sector 19 Rohini,Freehold,2.0,1,1.0,Semifurnished,28.731689,77.139148,1.20,925.000000,Old Property,High-End,Low Floor
K70358806,flat,North Delhi,Sector 19 Rohini,Freehold,2.0,2,1.0,Semifurnished,28.731689,77.139148,1.25,900.000000,Old Property,Basic,Low Floor
Q70358550,flat,North Delhi,Sector 19 Rohini,Freehold,2.0,2,1.0,Semifurnished,28.736416,77.140770,0.95,900.000000,Old Property,Basic,Low Floor
D70358376,flat,North Delhi,Sector 19 Rohini,Freehold,2.0,2,1.0,Semifurnished,28.736416,77.140770,1.05,900.000000,Old Property,Moderate,Low Floor
Q70358058,flat,North Delhi,Sector 19 Rohini,Freehold,2.0,2,1.0,Semifurnished,28.736982,77.140219,1.10,900.000000,Old Property,Moderate,Low Floor
M71198252,flat,North Delhi,Sector 19 Rohini,Freehold,2.0,2,1.0,Semifurnished,28.736982,77.140219,0.98,900.073475,Moderately old,Basic,Low Floor
F63477806,flat,North Delhi,Sector 19 Rohini,Freehold,2.0,2,1.0,Furnished,28.736982,77.140219,1.30,1000.000000,Old Property,High-End,Low Floor
R69260418,flat,North Delhi,Sector 19 Rohini,Freehold,2.0,2,2.0,Unfurnished,28.736982,77.140219,1.50,1000.000000,Old Property,Moderate,Low Floor
V69243576,house,North Delhi,Sector 19 Rohini,Freehold,10.0,4,2.0,Unfurnished,28.731689,77.139148,0.50,1000.000000,Moderately old,Basic,Mid Floor


In [ ]:
closest = distances_df.iloc[distances_df.index == 'K71113110'].T.reset_index().sort_values(by = 'K71113110', ascending = True)[0:10]["PROP_ID"].values

In [ ]:
df.iloc[df.index.isin(closest)]

,PROPERTY_TYPE,CITY_NAME,LOCALITY_NAME,OWNTYPE,BEDROOM_NUM,BATHROOM_NUM,BALCONY_NUM,FURNISH,LATITUDE,LONGITUDE,PRICE_CR,CALCULATED_AREA_SQFT,AGE_POSSESSION,AMENITIES_CLUSTER,FLOOR_CATEGORY
PROP_ID,,,,,,,,,,,,,,,
P70335066,flat,West Delhi,Dwarka Mor,Freehold,2.0,2,2.0,Semifurnished,28.733568,77.139694,1.05,1210.0,Moderately old,High-End,Mid Floor
K71113110,flat,North Delhi,Sector 19 Rohini,Freehold,2.0,1,1.0,Semifurnished,28.731689,77.139148,1.20,925.0,Old Property,High-End,Low Floor
A70361400,flat,North Delhi,Sector 18 Rohini,Freehold,2.0,2,0.0,Semifurnished,28.734830,77.135390,1.20,900.0,Old Property,Moderate,Low Floor
K70358806,flat,North Delhi,Sector 19 Rohini,Freehold,2.0,2,1.0,Semifurnished,28.731689,77.139148,1.25,900.0,Old Property,Basic,Low Floor
Q70358550,flat,North Delhi,Sector 19 Rohini,Freehold,2.0,2,1.0,Semifurnished,28.736416,77.140770,0.95,900.0,Old Property,Basic,Low Floor
D70358376,flat,North Delhi,Sector 19 Rohini,Freehold,2.0,2,1.0,Semifurnished,28.736416,77.140770,1.05,900.0,Old Property,Moderate,Low Floor
J71151526,flat,North Delhi,Sector 15 Rohini,Power of Attorney,3.0,2,2.0,Semifurnished,28.729535,77.133539,2.10,1280.0,Old Property,High-End,Low Floor
U71022292,flat,North Delhi,Sector 15 Rohini,Freehold,2.0,2,2.0,Semifurnished,28.732647,77.134847,1.15,1100.0,New Property,Basic,Mid Floor
Q69719280,flat,North Delhi,Sector 15 Rohini,Freehold,2.0,2,0.0,Semifurnished,28.732647,77.134847,1.05,1100.0,New Property,Basic,Mid Floor
